In [1]:
import numpy as np
from ipywidgets import IntSlider, FloatSlider, HBox, VBox
import bqplot.pyplot as plt
import pandas as pd
import math

In [2]:
df = pd.read_csv('Tabla16.csv', decimal=",")

In [3]:
def densidad(h):
    mayores = df.loc[df['Altitud m'] >= h]
    menores = df.loc[df['Altitud m'] <= h]
    mayor = mayores.head(1).iloc[0]
    menor = menores.tail(1).iloc[0]
    if mayor['Altitud m'] == menor['Altitud m']:
        return menor['Densidad kg/m^3']
    
    return menor['Densidad kg/m^3'] + (mayor['Densidad kg/m^3'] - menor['Densidad kg/m^3'])/(mayor['Altitud m'] - menor['Altitud m']) * (h - menor['Altitud m'])

def masa_total(masa_h, masa_p, pasajeros):
    return masa_h + masa_p * pasajeros

def potencia(masa, altura, diametro):
    d = densidad(altura)
    return math.sqrt(((masa * 9.81) ** 3)/(0.7 * d * math.pi * diametro**2))

In [4]:
x = np.linspace(0.0, 3000.0, 10)
potencia_vectorizada = np.vectorize(potencia)
y = potencia_vectorizada(masa_total(800, 20, 0), x, 5)

title_tmpl = 'Potencia necesaria para elevar un helicóptero a una altura h'
potencia_fig = plt.figure(title=title_tmpl.format(0, 1))
potencia_line = plt.plot(x, y, 'm', stroke_width=3)

In [5]:
masa_h = FloatSlider(description='Masa del helicóptero', min=800, max=5000, step=.1)
diametro = IntSlider(description='Diámetro de las aspas', min=5, max=20)
pasajeros = IntSlider(description='Número de pasajeres', min=0, max=12)
masa_p = FloatSlider(description='Masa promedio de pasajeres', min=20, max=150, step=.1)

slider_layout = HBox([masa_h, diametro, pasajeros, masa_p])

In [6]:
def update_potencia(change):
    new_masa_h = masa_h.value
    new_diametro = diametro.value
    new_pasajeros = pasajeros.value
    new_masa_p = masa_p.value
    potencia_line.y = potencia_vectorizada(masa_total(new_masa_h, new_masa_p, new_pasajeros), x, new_diametro)


masa_h.observe(update_potencia, 'value')
diametro.observe(update_potencia, 'value')
pasajeros.observe(update_potencia, 'value')
masa_p.observe(update_potencia, 'value')

In [7]:
final_layout = VBox([potencia_fig, slider_layout])
final_layout